In [ ]:
import os
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pysentiment2 as ps
import string

# 將sentence轉為token

In [ ]:
datapath = []
path = os.path.join(os.getcwd(), 'abbrev_news')
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
    for file in dir_path:
        if "_company.csv" in file:
            datapath.append(os.path.join(path, dirname, file))
print(len(datapath))
datapath[:5]

In [ ]:
for i in range(len(datapath)):
    print(i)
    path = datapath[i]
    data = pd.read_csv(path)


    data.drop_duplicates(subset=['headline'], inplace = True, ignore_index = True)#刪除重複sample
    data.drop(columns = ["link"], inplace = True)#刪除連結
    if 'time' in data.columns:
        data.drop(columns = ["time"], inplace = True)#後來抓得有些有時間


    data["head_token"] = None
    data["text_token"] = None


    stop_words = stopwords.words('english')#要刪掉的word
    exclude = set(string.punctuation)#要刪掉的標點符號
    for i in range(0, data.shape[0]):
        '''處理 Head'''
        #小寫
        text = data.iloc[i, 0].lower()
        # 移除標點符號
        text = ''.join(ch for ch in text if ch not in exclude)
        # token 句子
        tokens = word_tokenize(text)
        # 移除 stopwords
        tokens = [w for w in tokens if not w in stop_words]
        # 將 token 連成 str, 使用 , 連結
        data.iloc[i, -2] = ','.join(x for x in tokens)

        '''處理 text'''
        #小寫
        text = data.iloc[i, 2].lower()
        # 移除標點符號
        text = ''.join(ch for ch in text if ch not in exclude)
        # token 句子 
        tokens = word_tokenize(text)
        # 移除 stopwords
        tokens = [w for w in tokens if not w in stop_words]
        # 將 token 連成 str, 使用 , 連結
        data.iloc[i, -1] = ','.join(x for x in tokens)


    path = path.replace(".csv", "_b.csv")
    data.to_csv(path, index = False)#save


data.head()

# 計算各情緒字的個數

In [ ]:
datapath = []
path = os.path.join(os.getcwd(), 'abbrev_news')
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
    for file in dir_path:
        if ("_b.csv" in file):
            datapath.append(os.path.join(path, dirname, file))
print(len(datapath))
datapath[:5]

In [ ]:
for i in range(len(datapath)):
    print(i)
    path = datapath[i]
    b_data = pd.read_csv(path)


    b_data["head_pos"]      = 0
    b_data["head_neg"]      = 0
    b_data["head_neutral"]  = 0
    b_data["head_total"]    = 0
    b_data["text_pos"]      = 0
    b_data["text_neg"]      = 0
    b_data["text_neutral"]  = 0
    b_data["text_total"]    = 0


    hiv4 = ps.HIV4()
    for i in range(0, b_data.shape[0]):
        pos = 0
        neg = 0
        neutral = 0
        for word in b_data["head_token"][i].split(","):#用","把字切割成list
            tokens = hiv4.tokenize(word)#先把字放入hiv4.tokenize才能計算分數
            score = hiv4.get_score(tokens)#計算分數
            if(score["Polarity"] > 0):#正面
                pos = pos + 1
            elif (score["Polarity"] < 0) :#負面
                neg = neg + 1
            else:#中性
                neutral = neutral + 1
        b_data.iloc[i, -8] = pos
        b_data.iloc[i, -7] = neg
        b_data.iloc[i, -6] = neutral
        b_data.iloc[i, -5] = len(b_data["head_token"][i].split(","))#用","把自切割成list後計算長度-->總字數

        pos = 0
        neg = 0
        neutral = 0      
        for word in b_data["text_token"][i].split(","):#用","把字切割成list
            tokens = hiv4.tokenize(word)#先把字放入hiv4.tokenize才能計算分數
            score = hiv4.get_score(tokens)#計算分數
            if(score["Polarity"] > 0):#正面
                pos = pos + 1
            elif (score["Polarity"] < 0) :#負面
                neg = neg + 1
            else:#中性
                neutral = neutral + 1
        b_data.iloc[i, -4] = pos
        b_data.iloc[i, -3] = neg
        b_data.iloc[i, -2] = neutral
        b_data.iloc[i, -1] = len(b_data["text_token"][i].split(","))#用","把自切割成list後計算長度-->總字數


    path = path.replace("_b.csv", "_c.csv")
    b_data.to_csv(path, index = False)#save


b_data.head()

# 計算ratio

In [ ]:
datapath = []
path = os.path.join(os.getcwd(), 'abbrev_news')
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
    for file in dir_path:
        if ("_c.csv" in file):
            datapath.append(os.path.join(path, dirname, file))
print(len(datapath))
datapath[:5]

In [ ]:
for i in range(len(datapath)):
    print(i)
    path = datapath[i]
    c_data = pd.read_csv(path)


    aggregation_functions = {'head_pos': 'sum', 'head_neg': 'sum', 'head_neutral': "sum",\
                            'head_total': 'sum', 'text_pos': 'sum', 'text_neg': "sum",\
                            'text_neutral': 'sum', 'text_total': 'sum',}
    #按照日期groupby，並加總
    d_data = c_data.groupby(c_data['date']).aggregate(aggregation_functions)
    d_data.reset_index(inplace = True)


    d_data['head_Pos_ratio']     = None
    d_data['head_Neg_ratio']     = None
    d_data['head_Netural_ratio'] = None
    d_data['head_Polarity']      = None
    d_data['head_Subjectivity']  = None
    d_data['text_Pos_ratio']     = None
    d_data['text_Neg_ratio']     = None
    d_data['text_Netural_ratio'] = None
    d_data['text_Polarity']      = None
    d_data['text_Subjectivity']  = None


    for w in range(0, d_data.shape[0]):
        if(d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w]!=0):#分母不可為0
            d_data.iloc[w, 9] = d_data["head_pos"].iloc[w]\
            /(d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w])#head_Pos_ratio

            d_data.iloc[w, 10] = d_data["head_neg"].iloc[w]\
            /(d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w])#head_Neg_ratio

            d_data.iloc[w, 12] = (d_data["head_pos"].iloc[w]-d_data["head_neg"].iloc[w])\
            /(d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w])#head_Polarity
        else:
            d_data.iloc[w, 9] = 0 #head_Pos_ratio
            d_data.iloc[w, 10] = 0 #head_Neg_ratio        
            d_data.iloc[w, 12] = 0 #head_Polarity

        if(d_data["head_total"].iloc[w]!=0):#分母不可為0
            d_data.iloc[w, 11] = d_data["head_neutral"].iloc[w]/d_data["head_total"].iloc[w]#head_Netural_ratio

            d_data.iloc[w, 13] = (d_data["head_pos"].iloc[w]+d_data["head_neg"].iloc[w])/ \
            (d_data["head_total"].iloc[w])#head_Subjectivity
        else:
            d_data.iloc[w, 11] = 0 #head_Netural_ratio
            d_data.iloc[w, 13] = 0 #head_Subjectivity


        if(d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w]!=0):#分母不可為0
            d_data.iloc[w, 14] = d_data["text_pos"].iloc[w]\
            /(d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w])#text_Pos_ratio

            d_data.iloc[w, 15] = d_data["text_neg"].iloc[w]\
            /(d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w])#text_Neg_ratio

            d_data.iloc[w, 17] = (d_data["text_pos"].iloc[w]-d_data["text_neg"].iloc[w])\
            /(d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w])#text_Polarity     
        else:
            d_data.iloc[w, 14] = 0 #text_Pos_ratio
            d_data.iloc[w, 15] = 0 #text_Neg_ratio
            d_data.iloc[w, 17] = 0 #text_Polarity

        if(d_data["text_total"].iloc[w] != 0):#分母不可為0
            d_data.iloc[w, 16] =  d_data["text_neutral"].iloc[w]/d_data["text_total"].iloc[w]#text_Netural_ratio
            d_data.iloc[w, 18] = (d_data["text_pos"].iloc[w]+d_data["text_neg"].iloc[w])\
            /d_data["text_total"].iloc[w]#text_Subjectivity
        else:
            d_data.iloc[w, 16] = 0 #text_Netural_ratio
            d_data.iloc[w, 18] = 0 #text_Subjectivity


    path = path.replace("_c.csv", "_d.csv")
    d_data.to_csv(path, index = False)#save


d_data.head()

# 透過standarization計算ratio

In [ ]:
datapath = []
path = os.path.join(os.getcwd(), 'abbrev_news')
for dirname in os.listdir(path):#資料夾裡的子資料夾
    dir_path = os.listdir(os.path.join(path, dirname))
    for file in dir_path:
        if ("_d.csv" in file):
            datapath.append(os.path.join(path, dirname, file))
print(len(datapath))
datapath[:5]

In [ ]:
for i in range(len(datapath)):
    print(i)
    path = datapath[i]
    e_data = pd.read_csv(path)


    # standarization = (X - mean) / std
    e_data['head_sPos_ratio']     =\
    (e_data["head_Pos_ratio"]-e_data["head_Pos_ratio"].mean())/e_data["head_Pos_ratio"].std()

    e_data['head_sNeg_ratio']     =\
    (e_data["head_Neg_ratio"]-e_data["head_Neg_ratio"].mean())/e_data["head_Neg_ratio"].std()

    e_data['head_sNetural_ratio'] =\
    (e_data["head_Netural_ratio"]-e_data["head_Netural_ratio"].mean())/e_data["head_Netural_ratio"].std()

    e_data['head_sPolarity']      =\
    (e_data["head_Polarity"]-e_data["head_Polarity"].mean())/e_data["head_Polarity"].std()

    e_data['head_sSubjectivity']  =\
    (e_data["head_Subjectivity"]-e_data["head_Subjectivity"].mean())/e_data["head_Subjectivity"].std()

    e_data['text_sPos_ratio']     =\
    (e_data["text_Pos_ratio"]-e_data["text_Pos_ratio"].mean())/e_data["text_Pos_ratio"].std()

    e_data['text_sNeg_ratio']     =\
    (e_data["text_Neg_ratio"]-e_data["text_Neg_ratio"].mean())/e_data["text_Neg_ratio"].std()

    e_data['text_sNetural_ratio'] =\
    (e_data["text_Netural_ratio"]-e_data["text_Netural_ratio"].mean())/e_data["text_Netural_ratio"].std()

    e_data['text_sPolarity']      =\
    (e_data["text_Polarity"]-e_data["text_Polarity"].mean())/e_data["text_Polarity"].std()

    e_data['text_sSubjectivity']  =\
    (e_data["text_Subjectivity"]-e_data["text_Subjectivity"].mean())/e_data["text_Subjectivity"].std()
    
    
    #做standarization時，當std=0在分母，結果會是nan，須補0
    e_data.fillna(0, inplace = True)

    
    path = path.replace("_d.csv", "_e.csv")
    e_data.to_csv(path, index = False)#save

e_data.head()